Описание задачи
Дана матрица A:$$
A = \begin{pmatrix}
1 & 0 & 0\\
1 & 1 & 0\\
0 & -1 & 1000\\
\end{pmatrix}
$$

Необходимо найти $ A^n, $ где $ n \in \mathbb{N} $

Я думаю в задании вряд ли имелось в виду что просто надо написать 
np.linalg.matrix_power(A, P). Поэтому реализация возведения в степень за время 
$O(log(n))$ была подсмотрена в [этой статье](https://habr.com/ru/post/148336/).
Основной смысл ускорения процесса это то что мы раскладываем нашу степень на степени двойки то есть $ A^n=A^i*A^j*A^k*... $ где ($i+j+k+...=n$ при этом являются степенями двойки и $ i,j,k, ... \in \mathbb{N} $)  мы вычисляем $ A^{i/2} $ и перемножаем,для этого вычисляем $ A^{i/4} $ и так далее пока не дойдем до $ A^{i/i} $ и так же с каждым членом суммы j,k,... Еще следует упомянуть что $ A^n=A^i*A^j=A^j*A^i, \forall i,j \in \mathbb{N}$ натуральные степени матрицы перестановочны. В статье еще используется запоминание вычесленных степеней что бы избежать повторных вычеслений в соседних ветвях. Ниже приведена реализация.


In [ ]:
def multiply_matrix(M1, M2):
        """Умножение матриц
        (ожидаются матрицы в виде списка список размером 3x3)."""

        a11 = M1[0][0]*M2[0][0] + M1[0][1]*M2[1][0] + M1[0][2]*M2[2][0]
        a12 = M1[0][0]*M2[0][1] + M1[0][1]*M2[1][1] + M1[0][2]*M2[2][1]
        a13 = M1[0][0]*M2[0][2] + M1[0][1]*M2[1][2] + M1[0][2]*M2[2][2]
        
        a21 = M1[1][0]*M2[0][0] + M1[1][1]*M2[1][0] + M1[1][2]*M2[2][0]
        a22 = M1[1][0]*M2[0][1] + M1[1][1]*M2[1][1] + M1[1][2]*M2[2][1]
        a23 = M1[1][0]*M2[0][2] + M1[1][1]*M2[1][2] + M1[1][2]*M2[2][2]
        
        a31 = M1[2][0]*M2[0][0] + M1[2][1]*M2[1][0] + M1[2][2]*M2[2][0]
        a32 = M1[2][0]*M2[0][1] + M1[2][1]*M2[1][1] + M1[2][2]*M2[2][1]
        a33 = M1[2][0]*M2[0][2] + M1[2][1]*M2[1][2] + M1[2][2]*M2[2][2]
        r = [[a11, a12 , a13], [a21, a22 , a23], [a31, a32 , a33]]
        return r

In [ ]:
def power_2(n):  
  """ вспомогателльная функция раскладываем степень на степени двойки."""
  powers = []
  p = 0
  for digit in reversed(bin(n)):
      if digit == '1':
          powers.append(int(pow(2, p)))
      p += 1
  return powers

In [ ]:
def matrix_pow(M,n):
  """Основная функция возведения в степень"""
  memory_pow={}
  def matrix_2_power ( Matrix , n2p ):
    """Вспомогательная функция возведение в степень
    при условии что показатель является степенью двойки"""
    if n2p == 1:
      return Matrix
    elif n in memory_pow.keys():
      return memory_pow[n2p]
    else:
      half_rez = matrix_2_power(Matrix, int(n2p/2))
      res = multiply_matrix(half_rez, half_rez)
      memory_pow[n2p] = res
    return res
  result=[[1,0,0],[0,1,0],[0,0,1]]
  if n==0: 
    return result
  else:  
    for i in power_2(n):
      result = multiply_matrix ( result , matrix_2_power  ( M , i) )
    return result



Проверка функции:

In [ ]:
import numpy as np
for i in range(5):
  A=np.random.randint(0, 15, 9).reshape(3,3)
  p=np.random.randint(1, 15)
  print(np.linalg.matrix_power(A, p).all()==np.array(matrix_pow(A,p)).all())

True
True
True
True
True


Ответ:

In [ ]:
A=[[1,0,0],[1,1,0],[0,-1,1000]]
np.array(matrix_pow(A,7))


array([[1, 0, 0],
       [7, 1, 0],
       [-1002003004005006, -1001001001001001001, 1000000000000000000000]],
      dtype=object)

В общем виде ответ выглядит как то вот так:
$$
A = \begin{pmatrix}
1 & 0 & 0\\
n & 1 & 0\\
-(1002003...00(n-1)) & -1(001)х(n-1) & 1000^n\\
\end{pmatrix}
$$

Доказать что это общий вид можно по индукции перемножив $A^{n-1}$ и $A^n$